In [3]:
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import copy
from gtfparse import read_gtf
import os
import pyranges as pr


In [ ]:
#download most updated GTF file from ensembl

gtf_ = pr.read_gtf("/nfs/genomes/human_hg38_dec13_no_random/gtf/Homo_sapiens.GRCh38.99.chr.canonical.gtf") #dowload gtf file from ensembl
gtf_ = gtf_.df

In [16]:
genes = [ "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

gtf_xy = gtf_[gtf_.gene_name.isin(genes)]

#extract the exons and genes
gtf_xy = gtf_xy[(gtf_xy['Feature'] == "CDS")]

gtf_xy['Feature'].unique() 
gtf_xy['gene_name'].unique()
gtf_xy['Feature'].unique() 

array(['CDS'], dtype=object)

In [ ]:
def intron_overlapping(exons, intron_start, intron_end):
    
    #input data frame of exons, coordinates of intron_start and intron_ent 
    #if the intron overlaps an exon, returns false 

    for index in range(len(exons)): 
        exon_st = exons.iloc[index]['Start']
        exon_end = exons.iloc[index]['End']
        
        if ((exon_st >= intron_start) & (exon_st <= intron_end)):
            return True
        if ((exon_end >= intron_start) & (exon_end <= intron_end)):
            return True
        
        if ((exon_st <= intron_start) & (exon_end >= intron_start)):
            return True 
        if ((exon_st <= intron_end) & (exon_end >= intron_end)):
            return True

    return False


In [ ]:
intron_coordinates = pd.DataFrame(columns=['seqname', 'feature', 'start', 'end', 'gene_name'])  
genes = [ "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

#genes = ["ZFY"] 

for gene in genes: 
    testzfx = gtf_xy[gtf_xy['gene_name'] == gene] 
    exons = testzfx[testzfx['Feature'] == "CDS"]
    
    exons = exons.sort_values(['Chromosome', 'Start', 'End']) #sort values start to end 
    grouped_exons = exons.groupby('transcript_id')
    print(grouped_exons)
    for name, group in grouped_exons:
      #  print(name)
        for i in range(1, len(group)):
            intron_start = group.iloc[i-1][4] 
            intron_end = group.iloc[i][3] 
            
            if (intron_end - intron_start > 200) & (intron_end > intron_start): 
                intron_start = intron_start + 100
                intron_end = intron_end - 100
            else:
                continue
        # Check for overlap with exons of any transcripts 
            #get rid of any grouped exon 
            if (group.iloc[i-1][24] == '1') | (group.iloc[i][24] == '1'): 
                #print('FIRST', intron_start, intron_end)
                continue
            
            if intron_overlapping(exons, intron_start, intron_end):                 
                #print("HERE", intron_start, intron_end) 
                continue
            
            else:
                #print("adding", intron_start, intron_end) 
                new_row = pd.Series({'seqname': testzfx['Chromosome'].iloc[0], 'feature': 'intron', 'start': intron_start, 'end':intron_end, 'gene_name':gene}) 
                intron_coordinates = intron_coordinates.append(new_row, ignore_index=True) 
                

In [20]:
gtf_small = gtf_xy[['Chromosome', 'Feature', 'Start', 'End', 'gene_name']] 
gtf_small = gtf_small.rename(columns={"Chromosome": "seqname", "Feature": "feature", "Start":"start", "End": "end", "gene_name": "gene_name"}) #("seqname", "feature", "start", "end", "gene_name")
gtf_small[gtf_small['gene_name'] == 'DDX3X']

,seqname,feature,start,end,gene_name
2813677,chrX,CDS,41334252,41334297,DDX3X
2813680,chrX,CDS,41337407,41337465,DDX3X
2813682,chrX,CDS,41339035,41339083,DDX3X
2813684,chrX,CDS,41341483,41341616,DDX3X
2813686,chrX,CDS,41342494,41342653,DDX3X
...,...,...,...,...,...
2815274,chrX,CDS,41346319,41346410,DDX3X
2815276,chrX,CDS,41346504,41346622,DDX3X
2815278,chrX,CDS,41346858,41347012,DDX3X
2815280,chrX,CDS,41347311,41347451,DDX3X


In [21]:
vertically_concatenated = pd.concat([gtf_small, intron_coordinates], axis=0) 
vertically_concatenated = vertically_concatenated[(vertically_concatenated['feature'] == "CDS") | (vertically_concatenated['feature'] == "intron") | ((vertically_concatenated['feature'] == 'exon') )]

In [ ]:
vertically_concatenated['feature'].unique()
vertically_concatenated.to_csv(myPath + "/tables/" + "XY_gtf_w_introns_012924.txt",  index=None, sep=str('\t'))